**Installing required libraries**

In [ ]:
!pip install py2neo

     |████████████████████████████████| 177 kB 5.1 MB/s 


In [ ]:
!pip install openpyxl --upgrade

In [ ]:
!pip install neo4j

     |████████████████████████████████| 89 kB 3.6 MB/s 
  Created wheel for neo4j: filename=neo4j-4.4.1-py3-none-any.whl size=114783 sha256=40ae01c64a976931c7a1267b068deb7a375d68b29001d383170bfa9ae252f01a
  Stored in directory: /root/.cache/pip/wheels/fd/15/02/8379f87426cd1b74fd5891f49df9fb978423feb97650cd639b
Successfully built neo4j


In [ ]:
!pip install neo4jupyter

     |████████████████████████████████| 73 kB 1.1 MB/s 
  Created wheel for neo4jupyter: filename=neo4jupyter-0.2.0-py3-none-any.whl size=3869 sha256=8bd6e4890af8b23ed90abbc49b9e3f5bd012c2449524a9c8de1e069608c25087
  Stored in directory: /root/.cache/pip/wheels/98/e6/6e/04e4ab08495bd0e2bfeb0083436a6040c49f262c17d6345b2b
  Created wheel for ipython-cypher: filename=ipython_cypher-0.2.6-py3-none-any.whl size=13272 sha256=380f8d345cfceae16ea74a053367d06ca17bfa9156c755c97ac6b0041a2b1285
  Stored in directory: /root/.cache/pip/wheels/d9/ff/09/e790f902f0013f5479cf2b9288013d2b36f41da1877c7d4df2
  Created wheel for neo4jrestclient: filename=neo4jrestclient-2.1.1-py2.py3-none-any.whl size=78563 sha256=281a3a93f3f7b213af3d1df2e839c26ff5ec79ccd03be5bbdbc9a6282e4a51fa
  Stored in directory: /root/.cache/pip/wheels/4d/39/cd/03fa156cb423361fa55a1f69d711d152eb6f3ce902a324059e
Successfully built neo4jupyter ipython-cypher neo4jrestclient


**Importing Required Libraries**

In [ ]:
import pandas as pd

from neo4j import GraphDatabase, basic_auth
from py2neo import Graph

import re

import neo4jupyter

**Establishing a connection between our Neo4j database and python notebook**

In [ ]:
g = Graph("bolt://34.201.241.51:7687", password = "whirls-bullet-boils")

In [ ]:
driver = GraphDatabase.driver(
  "bolt://34.201.241.51:7687",
  auth=basic_auth("neo4j", "whirls-bullet-boils"))

# The uri "bolt://34.201.241.51:7687" needs to be replaced in case of new sandbox creation in neo4j.
# The credentials "neo4j" and "whirls-bullet-boils" also need replacement for each use case.

In [ ]:
def execute_transactions(transaction_execution_commands):
    # Establishing connection with database
    data_base_connection = GraphDatabase.driver( "bolt://34.201.241.51:7687",
  auth=basic_auth("neo4j", "whirls-bullet-boils"))
    # Creating a session
    session = data_base_connection.session()    
    for i in transaction_execution_commands:
        session.run(i)

**Loading the datasets**

In [ ]:
# This dataset consists of transactions which will be used to establish relationship between the customer and the stock.
df = pd.read_excel(r'Rec_sys_data-1.xlsx')

In [ ]:
# Little bit of preprocessing so that we can easily run NoSQL queries.
df['CustomerID'] = df['CustomerID'].apply(str)

In [ ]:
# This dataset contains detailed information about each stock which will be used to link stockcodes and their description/title. 
df1 = pd.read_excel('Rec_sys_data-1.xlsx','product')

In [ ]:
df1.head()

,StockCode,Product Name,Description,Category,Brand,Unit Price
0,22629,Ganma Superheroes Ordinary Life Case For Samsu...,"New unique design, great gift.High quality pla...",Cell Phones|Cellphone Accessories|Cases & Prot...,Ganma,13.99
1,21238,Eye Buy Express Prescription Glasses Mens Wome...,Rounded rectangular cat-eye reading glasses. T...,Health|Home Health Care|Daily Living Aids,Eye Buy Express,19.22
2,22181,MightySkins Skin Decal Wrap Compatible with Ni...,Each Nintendo 2DS kit is printed with super-hi...,Video Games|Video Game Accessories|Accessories...,Mightyskins,14.99
3,84879,Mediven Sheer and Soft 15-20 mmHg Thigh w/ Lac...,The sheerest compression stocking in its class...,Health|Medicine Cabinet|Braces & Supports,Medi,62.38
4,84836,Stupell Industries Chevron Initial Wall D cor,Features: -Made in the USA. -Sawtooth hanger o...,Home Improvement|Paint|Wall Decals|All Wall De...,Stupell Industries,35.99


**Uploading the entities in Neo4J Database**

To implement knowledge graphs in Neo4J we need to convert the dataframe into Relational Database. For this, we need to convert customers and stocks into entities (or nodes of a graph) and build a relation between them.

In [ ]:
# creating a list of all unique customer IDs 
customerids = df['CustomerID'].unique().tolist()

# storing all the create commands to be executed into create_customers list
create_customers = []

for i in customerids:
  # example of create statement "create (n:entity {property_key : '12345'})" 
  statement = "create (c:customer{cid:"+ '"' + str(i) + '"' +"})"
  create_customers.append(statement)

# running all the queries into neo4j to create customer entities
execute_transactions(create_customers)

Once we are done with customer nodes, we need to create nodes for stocks as well.

In [ ]:
# creating a lit of all unique stockcodes
stockcodes = df['StockCode'].unique().tolist()

# storing all the create commands to be executed into the create_stockcodes list
create_stockcodes = []

for i in stockcodes:
  # example of create statement "create (m:entity {property_key : 'XYZ'})"
  statement = "create (s:stock{stockcode:"+ '"' + str(i) + '"' +"})"
  create_stockcodes.append(statement)

# running all the queries into neo4j to create stock entities
execute_transactions(create_stockcodes)

Once we are done creating nodes for customers and stocks, we need to create a link between stockcodes and title which will be needed to recommend items.


For this we will create another property key called as 'title' into the stock entity already present in our neo4j database.

In [ ]:
# creating a blank dataframe
df2 = pd.DataFrame(columns = ['StockCode', 'Title'])

In [ ]:
# This cell of code will add all the unique stockcodes along with their title in df2
stockcodes = df['StockCode'].unique().tolist()

for i in range(len(stockcodes)):
  df2['StockCode'][i] = stockcodes[i]
  df2['Title'][i] = df1[df1['StockCode']==stockcodes[i]]['Product Name'].values

In [ ]:
# Doing some data preprocessing such that these queries can be run in neo4j
df2['Title'] = df2['Title'].apply(str)
df2['Title'] = df2['Title'].map(lambda x: re.sub(r'\W+', ' ', x))
df2['Title'] = df2['Title'].apply(str)

In [ ]:
# This query will add the 'title' property key to each stock entity in our neo4j database
for i in range(len(df2)):
  query = """
  MATCH (s:stock {stockcode:""" + '"' + str(df2['StockCode'][i]) + '"' + """})
  SET s.title ="""+ '"' + str(df2['Title'][i]) + '"' + """
  RETURN s.stockcode, s.title
  """

  g.run(query)

**Creating relation between customers and stocks**

Since we have all the transactions in our dataset, the relation is already known and present. But since we have to convert it into a RDS, we have to run cypher queries in neo4j to build this relationship.

In [ ]:
# Storing transaction values in a list
transaction_list = df.values.tolist()

# storing all commands to build relationship in an empty list relation
relation = []

for i in transaction_list:
  # the 9th column in df is customerID and 2nd column is stockcode which we are appending in the statement
  statement = """MATCH (a:customer),(b:stock) WHERE a.cid = """+'"' + str(i[8])+ '"' + """ AND b.stockcode = """ + '"' + str(i[1]) + '"' + """ CREATE (a)-[:bought]->(b) """
  relation.append(statement)

In [ ]:
execute_transactions(relation)

**Finding similarity between users using the relationship created**

In [ ]:
def similar_users(id) : 
  # This query will find users who have bought stocks in common with the customer having id specified by user 
  # Later we will find jaccard index for each of them 
  # We wil return the neighbors sorted by jaccard index in descending order
  query = """
  MATCH (c1:customer)-[:bought]->(s:stock)<-[:bought]-(c2:customer)
  WHERE c1 <> c2 AND c1.cid =""" + '"' + str(id) +'"' """
  WITH c1, c2, COUNT(DISTINCT s) as intersection
  MATCH (c:customer)-[:bought]->(s:stock)
  WHERE c in [c1, c2]
  WITH c1, c2, intersection, COUNT(DISTINCT s) as union
  WITH c1, c2, intersection, union, (intersection * 1.0 / union) as jaccard_index
  ORDER BY jaccard_index DESC, c2.cid
  WITH c1, COLLECT([c2.cid, jaccard_index, intersection, union])[0..15] as neighbors
  WHERE SIZE(neighbors) = 15   // return users with enough neighbors
  RETURN c1.cid as customer, neighbors
  
  """
  neighbors = pd.DataFrame([['CustomerID','JaccardIndex','Intersection','Union']])
  for i in g.run(query).data():
    neighbors = neighbors.append(i["neighbors"])
  
  print("\n----------- customer's 15 nearest neighbors ---------\n")
  print(neighbors)
  

*Sample Outputs*

In [ ]:
similar_users('12347')


----------- customer's 15 nearest neighbors ---------

             0             1             2      3
0   CustomerID  JaccardIndex  Intersection  Union
0        17396      0.111111            10     90
1        13821      0.108333            13    120
2        17097      0.107784            18    167
3        13324      0.103093            10     97
4        15658      0.099099            11    111
5        15606      0.097345            11    113
6        16389       0.09375             9     96
7        18092      0.092784             9     97
8        13814      0.091743            10    109
9        13265       0.08871            11    124
10       13488      0.087248            26    298
11       12843      0.086957            12    138
12       16618      0.086207            10    116
13       15502      0.084821            19    224
14       17722       0.08427            15    178


In [ ]:
similar_users(17975)


----------- customer's 15 nearest neighbors ---------

             0             1             2      3
0   CustomerID  JaccardIndex  Intersection  Union
0        15356      0.131098            43    328
1        18231      0.126531            31    245
2        14395      0.125436            36    287
3        15856      0.124668            47    377
4        16907      0.124424            27    217
5        17787      0.123404            29    235
6        15059       0.11236            30    267
7        13344       0.11215            24    214
8        16222      0.111111            29    261
9        17085      0.108949            28    257
10       17450       0.10687            28    262
11       17865      0.106796            33    309
12       16910      0.106061            35    330
13       16549      0.105919            34    321
14       13263      0.105155            51    485


In [ ]:
similar_users(16359)


----------- customer's 15 nearest neighbors ---------

             0             1             2      3
0   CustomerID  JaccardIndex  Intersection  Union
0        15319      0.148936             7     47
1        15124      0.146341             6     41
2        13706      0.137255             7     51
3        15985      0.135593             8     59
4        12842      0.130435             6     46
5        14226      0.130435             9     69
6        14651      0.127907            11     86
7        14959      0.126984             8     63
8        13900      0.126761             9     71
9        16989         0.125             5     40
10       14209      0.121951            10     82
11       16721      0.119403             8     67
12       15076      0.115385             9     78
13       15643      0.112903             7     62
14       15681      0.111111             6     54


In [ ]:
similar_users(17867)


----------- customer's 15 nearest neighbors ---------

             0             1             2      3
0   CustomerID  JaccardIndex  Intersection  Union
0        13551           0.2             2     10
1        16566      0.133333             2     15
2        12917      0.111111             1      9
3        13967      0.111111             1      9
4        14332      0.111111             1      9
5        15442      0.111111             1      9
6        17483      0.105263             2     19
7        13217           0.1             1     10
8        17384           0.1             1     10
9        16526      0.095238             2     21
10       17854      0.095238             2     21
11       15160      0.090909             1     11
12       13207      0.083333             1     12
13       18032      0.083333             1     12
14       13805      0.076923             1     13


**Recommending items based on the similar users**

In [ ]:
def recommend(id):
  # The query below is same as similar_users function
  # It will return the most similar customers
  query1 = """
  MATCH (c1:customer)-[:bought]->(s:stock)<-[:bought]-(c2:customer)
  WHERE c1 <> c2 AND c1.cid =""" + '"' + str(id) +'"' """
  WITH c1, c2, COUNT(DISTINCT s) as intersection
  MATCH (c:customer)-[:bought]->(s:stock)
  WHERE c in [c1, c2]
  WITH c1, c2, intersection, COUNT(DISTINCT s) as union
  WITH c1, c2, intersection, union, (intersection * 1.0 / union) as jaccard_index
  ORDER BY jaccard_index DESC, c2.cid
  WITH c1, COLLECT([c2.cid, jaccard_index, intersection, union])[0..15] as neighbors
  WHERE SIZE(neighbors) = 15   // return users with enough neighbors
  RETURN c1.cid as customer, neighbors
  
  """
  neighbors = pd.DataFrame([['CustomerID','JaccardIndex','Intersection','Union']])
  neighbors_list = {}
  for i in g.run(query1).data():
    neighbors = neighbors.append(i["neighbors"])
    neighbors_list[i["customer"]] = i["neighbors"]
    print(neighbors_list)
    
  # From the neighbors_list returned, we will fetch the customer ids of those neighbors to recommend items
  nearest_neighbors = [neighbors_list[id][i][0] for i in range(len(neighbors_list[id]))]

  # The below query will fetch all the items boughts by nearest neighbors
  # We will remove the items which have been already bought by the target customer
  # Now from the filtered set of items, we will count how many times each item is repeating within the shopping carts of nearest neighbors
  # We will sort that list on count of repititions and return in descending order
  query2 = """
        // get top n recommendations for customer from their nearest neighbors
        MATCH (c1:customer),(neighbor:customer)-[:bought]->(s:stock)    // all items bought by neighbors
        WHERE c1.cid = """ + '"' + str(id) + '"' """ 
          AND neighbor.cid in $nearest_neighbors
          AND not (c1)-[:bought]->(s)                    // filter for items that our user hasn't bought before
        
        WITH c1, s, COUNT(DISTINCT neighbor) as countnns // times bought by nearest neighbors
        ORDER BY c1.cid, countnns DESC               
        RETURN c1.cid as customer, COLLECT([s.title, s.stockcode, countnns])[0..$n] as recommendations  
        """

  recommendations = pd.DataFrame([['Title','StockCode','Number of times bought by neighbors']])
  for i in g.run(query2, id = id, nearest_neighbors = nearest_neighbors, n = 5).data():
    #recommendations[i["customer"]] = i["recommendations"]
    recommendations = recommendations.append(i["recommendations"])


  # We will also print the items bought earlier by the target customer
  print(" \n---------- Top 8 StockCodes bought by customer " + str(id) + " -----------\n")
  
  print(df[df['CustomerID']==id][['CustomerID','StockCode','Quantity']].nlargest(8,'Quantity'))

  bought = df[df['CustomerID']==id][['CustomerID','StockCode','Quantity']].nlargest(8,'Quantity')

  print('\n-------Product Name of bought StockCodes ------\n')

  print((df1[df1.StockCode.isin(bought.StockCode)]['Product Name']).to_string())

  # Here we will print the recommendations
  print("------------ \n Recommendations for Customer {} ------- \n".format(id))
  print(recommendations.to_string())


In [ ]:
recommend('17850')

{'17850': [['15497', 0.14814814814814814, 4, 27], ['17169', 0.14705882352941177, 5, 34], ['18170', 0.14285714285714285, 6, 42], ['15636', 0.1388888888888889, 5, 36], ['13187', 0.13725490196078433, 7, 51], ['15722', 0.1276595744680851, 6, 47], ['15482', 0.11764705882352941, 4, 34], ['13161', 0.11538461538461539, 3, 26], ['14035', 0.11428571428571428, 4, 35], ['17866', 0.1111111111111111, 3, 27], ['13884', 0.10714285714285714, 6, 56], ['14440', 0.10638297872340426, 5, 47], ['12747', 0.1, 5, 50], ['17742', 0.0967741935483871, 3, 31], ['14163', 0.09615384615384616, 5, 52]]}
 
---------- Top 8 StockCodes bought by customer 17850 -----------

     CustomerID StockCode  Quantity
285       17850    82494L        12
2629      17850    85123A        12
2634      17850     71053        12
2978      17850     71053        12
2983      17850    85123A        12
3299      17850     71053        12
3301      17850     21068        12
3302      17850    84029G        12

-------Product Name of bought 

In [ ]:
recommend("16359")

{'16359': [['15319', 0.14893617021276595, 7, 47], ['15124', 0.14634146341463414, 6, 41], ['13706', 0.13725490196078433, 7, 51], ['15985', 0.13559322033898305, 8, 59], ['12842', 0.13043478260869565, 6, 46], ['14226', 0.13043478260869565, 9, 69], ['14651', 0.12790697674418605, 11, 86], ['14959', 0.12698412698412698, 8, 63], ['13900', 0.1267605633802817, 9, 71], ['16989', 0.125, 5, 40], ['14209', 0.12195121951219512, 10, 82], ['16721', 0.11940298507462686, 8, 67], ['15076', 0.11538461538461539, 9, 78], ['15643', 0.11290322580645161, 7, 62], ['15681', 0.1111111111111111, 6, 54]]}
 
---------- Top 8 StockCodes bought by customer 16359 -----------

       CustomerID StockCode  Quantity
161089      16359     84836       120
232132      16359     84836        72
101750      16359     84836        60
124252      16359     84836        60
53524       16359    16169M        50
75394       16359     84836        48
33114       16359     84836        36
53526       16359     84836        36

------

In [ ]:
recommend("17867")

{'17867': [['13551', 0.2, 2, 10], ['16566', 0.13333333333333333, 2, 15], ['12917', 0.1111111111111111, 1, 9], ['13967', 0.1111111111111111, 1, 9], ['14332', 0.1111111111111111, 1, 9], ['15442', 0.1111111111111111, 1, 9], ['17483', 0.10526315789473684, 2, 19], ['13217', 0.1, 1, 10], ['17384', 0.1, 1, 10], ['16526', 0.09523809523809523, 2, 21], ['17854', 0.09523809523809523, 2, 21], ['15160', 0.09090909090909091, 1, 11], ['13207', 0.08333333333333333, 1, 12], ['18032', 0.08333333333333333, 1, 12], ['13805', 0.07692307692307693, 1, 13]]}
 
---------- Top 8 StockCodes bought by customer 17867 -----------

       CustomerID StockCode  Quantity
137223      17867    18097C       288
188192      17867     23109        42
188161      17867     85103        37
64167       17867     21633         2
188162      17867    15056N         2
137222      17867     21631         1
137224      17867    72351B         1
137225      17867    47599A         1

-------Product Name of bought StockCodes ------


In [ ]:
recommend("12347")

{'12347': [['17396', 0.1111111111111111, 10, 90], ['13821', 0.10833333333333334, 13, 120], ['17097', 0.10778443113772455, 18, 167], ['13324', 0.10309278350515463, 10, 97], ['15658', 0.0990990990990991, 11, 111], ['15606', 0.09734513274336283, 11, 113], ['16389', 0.09375, 9, 96], ['18092', 0.09278350515463918, 9, 97], ['13814', 0.09174311926605505, 10, 109], ['13265', 0.08870967741935484, 11, 124], ['13488', 0.087248322147651, 26, 298], ['12843', 0.08695652173913043, 12, 138], ['16618', 0.08620689655172414, 10, 116], ['15502', 0.08482142857142858, 19, 224], ['17722', 0.08426966292134831, 15, 178]]}
 
---------- Top 8 StockCodes bought by customer 12347 -----------

       CustomerID StockCode  Quantity
99443       12347     23076       240
10526       12347     22492        36
99444       12347     22492        36
153949      12347     17021        36
200488      12347    84558A        36
10527       12347    85167B        30
10534       12347    84558A        24
43446       12347     8